In [1]:
import pandas as pd
import numpy as np
np.random.seed(1)

In [2]:
df = pd.util.testing.makeTimeDataFrame()

/workspace/.pyenv_mirror/user/current/lib/python3.8/site-packages/pandas/util/__init__.py:16: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing


In [3]:
df

,A,B,C,D
2000-01-03,1.624345,-0.691661,-0.754398,1.198918
2000-01-04,-0.611756,-0.396754,1.252868,0.185156
2000-01-05,-0.528172,-0.687173,0.512930,-0.375285
2000-01-06,-1.072969,-0.845206,-0.298093,-0.638730
2000-01-07,0.865408,-0.671246,0.488518,0.423494
2000-01-10,-2.301539,-0.012665,-0.075572,0.077340
2000-01-11,1.744812,-1.117310,1.131629,-0.343854
2000-01-12,-0.761207,0.234416,1.519817,0.043597
2000-01-13,0.319039,1.659802,2.185575,-0.620001
2000-01-14,-0.249370,0.742044,-1.396496,0.698032


In [ ]:
df.to_csv('0323.csv')

预期结果：

第一行为：

In [4]:
result_1 = df.iloc[:20].corr(method='spearman').mean()
result_1.name = df.index[19]
result_1 = result_1.to_frame().T
result_1

,A,B,C,D
2000-01-28,0.239098,0.283083,0.262406,0.098872


第二行为：

In [5]:
result_2 = df.iloc[1:21].corr(method='spearman').mean()
result_2.name = df.index[20]
result_2 = result_2.to_frame().T
result_2

,A,B,C,D
2000-01-31,0.288722,0.287594,0.321429,0.137594
